In [ ]:
!pip install accelerate peft bitsandbytes transformers trl tqdm pandas torch transformers

In [2]:
import pandas as pd
import torch
import pandas as pd
from tqdm import tqdm

In [3]:
def format_prompt_test(essay, assignment):
    instructions = (
        "You are my English teacher. Read my essay and assignment."
        "Give me feedback to help me revise. Extract three to five very short excerpts from my essay and give me feedback on those. Keep your feedback very short."
        "List the excerpts and feedback like this:\n"
        "***[excerpt]---[feedback]\n"
        "***[excerpt]---[feedback]\n"
        "***[excerpt]---[feedback]\n"
    )
    return f"### INSTRUCTIONS: {instructions}\n ESSAY: {essay}\n ASSIGNMENT: {assignment}\n ### FEEDBACK:"

In [4]:
# Load dataset
df = pd.read_csv('eval_data_asap.csv')
df['input'] = df.apply(lambda row: format_prompt_test(row['essay'], row['prompt']), axis=1)

In [5]:
df.head()

,essay_id,prompt,essay,grade_level,domain1_score,type,input
0,1206,"More and more people use computers, but not ev...","Dear editor @ORGANIZATION2 the source, @CAPS1,...",8,11,persuasive,### INSTRUCTIONS: You are my English teacher. ...
1,12191,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....",The mood of the memoir is appreciation. I thin...,8,3,litanalysis,### INSTRUCTIONS: You are my English teacher. ...
2,13093,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....","In the excerpt ""Narciso Rodriguez"" from the bo...",8,4,litanalysis,### INSTRUCTIONS: You are my English teacher. ...
3,1317,"More and more people use computers, but not ev...","Dear local Newspaper, @CAPS1 you love computer...",8,8,persuasive,### INSTRUCTIONS: You are my English teacher. ...
4,13619,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....",The @CAPS1 sets a good mood in this memoir abo...,8,3,litanalysis,### INSTRUCTIONS: You are my English teacher. ...


In [6]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("meiflwr/llama2-7b-feedback-ft")
base_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
model = PeftModel.from_pretrained(base_model, "meiflwr/llama2-7b-feedback-ft")
tokenizer = AutoTokenizer.from_pretrained("meiflwr/llama2-7b-feedback-ft")

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/mxtan/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:51<00:00, 25.79s/it]


In [7]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 200, device = 0)
result = pipe(f"{df['input'][0]}")

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [8]:
print(result[0]['generated_text'])

### INSTRUCTIONS: You are my English teacher. Read my essay and assignment.Give me feedback to help me revise. Extract three to five very short excerpts from my essay and give me feedback on those. Keep your feedback very short.List the excerpts and feedback like this:
***[excerpt]---[feedback]
***[excerpt]---[feedback]
***[excerpt]---[feedback]

 ESSAY: Dear editor @ORGANIZATION2 the source, @CAPS1, I have noticed that many people have been spending too much time using their computers. This is something that needs to stop because it imports peoples health, safety and education. To open my statements, overuse @ORGANIZATION2 the computer con lead to poor health. Take my friend @PERSON1 for example. He sits at home on his computer all @DATE1 and plays games, goes on @CAPS2 and @CAPS3. He rarely gets any excersize and he almost never sees daylight. @PERSON1 @LOCATION1 gained @NUM1 pounds, which lead to obeisity and heart problems. He even wears wooden teeth because his general hygene decl

In [9]:
from transformers import pipeline
def generate(input, model, tokenizer):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 200, device = 0)
    result = pipe(f"{input}")
    res = result[0]['generated_text']
    return res

In [10]:
from tqdm import tqdm
tqdm.pandas()
df['predictions'] = df.progress_apply(lambda row: generate(row['input'], model, tokenizer), axis=1)

100%|██████████| 55/55 [09:16<00:00, 10.11s/it]


In [11]:
import re
def extract_text_after_tag(text, tag):
    pattern = re.compile(f"{re.escape(tag)}(.*)", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1).strip()
    return ""

def extract_feedback_sections(text):
    # Split the text by asterisks to separate sections
    sections = text.split('***')[1:]  # Ignore the part before the first asterisk
    
    result = []
    for section in sections:
        # Use regex to extract the text between * and ---
        excerpt_match = re.search(r'"(.*?)"---', section, re.DOTALL)
        excerpt = excerpt_match.group(1).strip() if excerpt_match else ''

        # Extract the feedback text after --- and before \n
        feedback_match = re.search(r'---(.*?)(\n|$)', section, re.DOTALL)
        #feedback_match = re.search(r'---(.*)', section, re.DOTALL)
        feedback = feedback_match.group(1).strip() if feedback_match else ''

        if len(excerpt) > 0 and len(feedback) > 0:
            result.append({'excerpt': excerpt, 'feedback': feedback})
    
    return result

def expand_pred_df(df, input_column):
    new_rows = []
    for idx, row in df.iterrows():
        input_text = row[input_column]
        sections = extract_feedback_sections(input_text)
        for section in sections:
            new_row = row.to_dict()
            new_row['excerpt'] = section['excerpt']
            new_row['feedback'] = section['feedback']
            new_rows.append(new_row)
    
    expanded_df = pd.DataFrame(new_rows)
    return expanded_df

In [12]:
df['extracted_content'] = df['predictions'].apply(lambda input_text: extract_text_after_tag(input_text, tag = "### FEEDBACK:"))


In [13]:
df.head()

,essay_id,prompt,essay,grade_level,domain1_score,type,input,predictions,extracted_content
0,1206,"More and more people use computers, but not ev...","Dear editor @ORGANIZATION2 the source, @CAPS1,...",8,11,persuasive,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""safety is on bigger issue""---I agree, but ..."
1,12191,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....",The mood of the memoir is appreciation. I thin...,8,3,litanalysis,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***"" The mood of the memoir is appreciation. I..."
2,13093,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....","In the excerpt ""Narciso Rodriguez"" from the bo...",8,4,litanalysis,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""The use of the word ""warm"" creates the bri..."
3,1317,"More and more people use computers, but not ev...","Dear local Newspaper, @CAPS1 you love computer...",8,8,persuasive,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""Are you for or against this descison of pe..."
4,13619,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....",The @CAPS1 sets a good mood in this memoir abo...,8,3,litanalysis,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""The @CAPS1 explains the mood by a Cuban fe..."


In [14]:
res_df = expand_pred_df(df, 'extracted_content')
res_df.head()

,essay_id,prompt,essay,grade_level,domain1_score,type,input,predictions,extracted_content,excerpt,feedback
0,1206,"More and more people use computers, but not ev...","Dear editor @ORGANIZATION2 the source, @CAPS1,...",8,11,persuasive,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""safety is on bigger issue""---I agree, but ...",safety is on bigger issue,"I agree, but what can we do to address the saf..."
1,1206,"More and more people use computers, but not ev...","Dear editor @ORGANIZATION2 the source, @CAPS1,...",8,11,persuasive,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""safety is on bigger issue""---I agree, but ...",The massive amounts @ORGANIZATION2 typing on t...,Good job providing evidence!
2,1206,"More and more people use computers, but not ev...","Dear editor @ORGANIZATION2 the source, @CAPS1,...",8,11,persuasive,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""safety is on bigger issue""---I agree, but ...","My bro, @PERSON2, had on assignment to do @DAT...","Brother, how could we strengthen our argument?..."
3,1206,"More and more people use computers, but not ev...","Dear editor @ORGANIZATION2 the source, @CAPS1,...",8,11,persuasive,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***""safety is on bigger issue""---I agree, but ...","In closing, overusing your computer can cause ...","Excellent summary! Again, I would make sure to..."
4,12191,"Read ""Narciso Rodriguez"" by Narciso Rodriguez....",The mood of the memoir is appreciation. I thin...,8,3,litanalysis,### INSTRUCTIONS: You are my English teacher. ...,### INSTRUCTIONS: You are my English teacher. ...,"***"" The mood of the memoir is appreciation. I...",The mood of the memoir is appreciation. I thin...,Good job identifying the mood of the memoir. R...


In [15]:
res_df.shape

(200, 11)

In [17]:
res_df.to_csv('finetune_res_asap.csv', index=False)